## Old

### Importieren der Klassen

In [ ]:
import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb
import accelerate

### Festlegen des Modells

In [ ]:
model_id="/home/thsch026/masterarbeit/models/generated/llm-awq/Meta-Llama-3-8B-Instruct-HF-AWQ-4bit"

### Initialisieren des Modells

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=quantization_config, #load_in_8bit=True,#
    device_map='auto',
    token='<EOL>'
)
#peft_model = "BojanaBas/Meta-Llama-3-8B-Instruct-pqa-10"
#model = PeftModel.from_pretrained(base_model, peft_model)

### Inferenz Test

### Autoawq library is needed to test inference with AWQ Models

In [ ]:
pip install autoawq

In [ ]:
# Eingabetext
input_text = "Once upon a time"
#input_text = "Why should I eat healthy?"

# Tokenisiere den Eingabetext
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Messe die Zeit vor der Inferenz
start_time = time.time()

# Führe die Inferenz durch
outputs = model.generate(input_ids, max_length=50)

# Messe die Zeit nach der Inferenz
end_time = time.time()

# Berechne die Inferenz-Zeit
inference_time = end_time - start_time

# Dekodiere die Ausgabe
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
number_of_words = len(output_text.split())
number_of_tokens = len(outputs[0])

token_per_second = number_of_tokens/inference_time

print(f"Inferenz-Zeit: {inference_time} Sekunden")
print(f"Output: {output_text}")
print("Tokens per second: ", token_per_second)



In [ ]:
import csv

# Die neuen Daten, die hinzugefügt werden sollen
neue_daten = ['Lukas', 28]

# Öffne die Datei im Anhangsmodus ('a' steht für append)
with open('daten.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    
    # Schreibe die neuen Daten in die CSV-Datei
    writer.writerow(neue_daten)

print("Neue Daten erfolgreich hinzugefügt.")

In [ ]:
pip install sentencepiece

## Evaluierung Modelle

### Sentencepiece muss ggf. noch mal installiert werden für Mistral (fast tokenizer)

In [ ]:
pip install sentencepiece

In [ ]:
!pip3 install transformers optimum autoawq bitsandbytes sentencepiece torch time accelerate csv packaging
# !pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

### Laden der libaries und festlegen der GPU

In [1]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
!echo $CUDA_VISIBLE_DEVICES
# torch.cuda.set_device(1)

0,1,2,3


In [ ]:
!pip install optimum

In [2]:
import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig 
import bitsandbytes as bnb
import accelerate
import csv
import packaging
#from awq import AutoAWQForCausalLM

### Ergebnis Dateien

In [4]:
full_csv= "/home/thsch026/masterarbeit/Slim/results/inference/inference12_full.csv"
data_csv= "/home/thsch026/masterarbeit/Slim/results/inference/inference12_memory_mean.csv"

##  1. Referenz Modelle

### Llama-3-8B-Instruct

In [20]:
model_id ="Meta-Llama-3-8B-Instruct"
model_loc = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = "meta-llama/Meta-Llama-3-8B-Instruct"

### Mistral-7B-Instruct-v0.2

In [60]:
model_id = "Mistral-7B-Instruct-v0.2"
model_loc = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = model_loc

### Llama-2-7b-chat-hf

In [8]:
model_loc = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = "meta-llama/Llama-2-7b-chat-hf"
model_id = "Llama-2-7b-chat"

## 2. AWQ Modelle

### Llama-3-8B-Instruct-AWQ-4bit

In [8]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Meta-Llama-3-8B-Instruct-AWQ"
model_loc = model_path + model_id
tokenizer = model_loc

### Mistral-7B-Instruct-v0.2-AWQ-4bit

In [63]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Mistral-7B-Instruct-v0.2-AWQ-4bit"
model_loc = model_path + model_id
tokenizer = model_path + model_id

### Llama-2-7b-chat-HF-AWQ-4bit

In [7]:
model_loc = "/home/thsch026/masterarbeit/models/generated/llm-awq/Llama-2-7b-chat-HF-AWQ-4bit"
tokenizer = "/home/thsch026/masterarbeit/models/generated/llm-awq/Llama-2-7b-chat-HF-AWQ-4bit"
model_id = "Llama-2-7b-chat-HF-AWQ-4bit"

## 3. Pruned Models (shortened-LLM)

### Meta-Llama-3-8B-Instruct

In [41]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Meta-LLama-3-8B-instruct"
model_id = "Meta-Llama-3-8B-Instruct_pruned(llm-shortened)"
tokenizer = model_loc

### Mistral-7B-Instruct-v02

In [66]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Mistral-7B-Instruct-v02"
model_id = "Mistral-7B-Instruct-v02_pruned(llm-shortened)"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Mistral-7B-Instruct-v02"

### Llama-2-7b-chat-hf pruned six layers

In [10]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Llama-2-7b-hf"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Llama-2-7b-hf"
model_id = "Llama-2-7b-hf_pruned_llm-shortened"

## 3 a. Pruned Models PruneMe

#### Meta-Llama-3-8B-Instruct-8_Ext

In [81]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/pruneme/Meta-Llama-3-8B-Instruct-8_Ext"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune/pruneme/Meta-Llama-3-8B-Instruct-8_Ext"
model_id = "Meta-Llama-3-8B-Instruct-8_Ext"

#### Meta-Llama-3-8B-Instruct-12_ext

In [84]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/pruneme/Meta-Llama-3-8B-Instruct-12_Ext"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune/pruneme/Meta-Llama-3-8B-Instruct-12_Ext"
model_id = "Meta-Llama-3-8B-Instruct-12_Ext"

#### Mistral 7 B Instruct v02 8 layers pruned

In [8]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/pruneme/Mistral-7B-Instruct-v0.2-8_Ext"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune/pruneme/Mistral-7B-Instruct-v0.2-8_Ext"
model_id = "Mistral-7B-Instruct-v0.2-8_Ext"

#### Mistral 7 B Instruct v02 12 layers pruned

In [11]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/pruneme/Mistral-7B-Instruct-v0.2-12_Ext"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune/pruneme/Mistral-7B-Instruct-v0.2-12_Ext"
model_id = "Mistral-7B-Instruct-v0.2-12_Ext"

## 4. Lora Models

#### Meta-Llama-3-8B-Instruct_ms-marco - ok

In [44]:
model_loc = "/home/thsch026/masterarbeit/models/generated/lora/Llama-3-8B-Instruct_ms-marco"
model_id = "Llama-3-8B-Instruct_ms-marco"
tokenizer = "meta-llama/Meta-Llama-3-8B-Instruct"

#### Mistral-7B-Instruct-v02_ms-marco

In [69]:
model_loc = "/home/thsch026/masterarbeit/models/generated/lora/mistral_7B-Instruct-v02_ms-marco"
tokenizer = "/home/thsch026/masterarbeit/models/generated/lora/mistral_7B-Instruct-v02_ms-marco"
model_id = "mistral_7B-Instruct-v02_ms-marco"

#### Llama-2-7b-chat_ms-marco

In [13]:
model_loc = "/home/thsch026/masterarbeit/models/generated/lora/Llama-2-7b-chat_ms-marco"
tokenizer = "/home/thsch026/masterarbeit/models/generated/lora/Llama-2-7b-chat_ms-marco"
model_id = "Llama-2-7b-chat_ms-marco"

##  5. Qlora Models

### Meta-Llama-3-8B-Instruct_qlora

In [11]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/Meta-Llama-3-8B-Instruct_qlora/merged"
model_id = "Meta-Llama-3-8B-Instruct_qlora"
tokenizer = "/home/thsch026/masterarbeit/models/generated/qlora/Meta-Llama-3-8B-Instruct_qlora"

### Mistral-7B-Instruct-v0.2_qlora

In [14]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/Mistral-7B-Instruct-v0.2_qlora/merged"
tokenizer = "/home/thsch026/masterarbeit/models/generated/qlora/Mistral-7B-Instruct-v0.2_qlora"
model_id = "Mistral-7B-Instruct-v0.2_qlora"

### llama-2-7b-chat_qlora

In [5]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/Llama-2-7b-chat-hf_qlora/merged"
tokenizer = "/home/thsch026/masterarbeit/models/generated/qlora/Llama-2-7b-chat-hf_qlora"
model_id = "llama-2-7b-chat_qlora"

## 6. Prune + Lora Models (shortened-LLM)

#### Meta-Llama-3-8B-Instruct -> Pruned + Lora

In [50]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Meta-Llama-3-8B-instruct_prune_lora"
model_id = "Meta-Llama-3-8B-instruct_prune_lora"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Meta-Llama-3-8B-instruct_prune_lora"

#### Mistral-7B-Instruct-v02_prune_lora

In [75]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Mistral-7B-Instruct-v02_prune_lora"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Mistral-7B-Instruct-v02_prune_lora"
model_id = "Mistral-7B-Instruct-v02_prune_lora"

#### Llama-2-7b-hf_prune_lora

In [19]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Llama-2-7b-hf_prune_lora"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Llama-2-7b-hf_prune_lora"
model_id = "Llama-2-7b-hf_prune_lora"

## 7. Prune + Lora + AWQ Models

#### Mistral-7B-Instruct-v02_prune_lora_awq

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Mistral-7B-Instruct-v02_prune_lora_awq"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Mistral-7B-Instruct-v02_prune_lora_awq"
model_id = "Mistral-7B-Instruct_prune_lora_awq"

#### Meta-Llama-3-8B-Instruct_prune_lora_awq

In [53]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Meta-Llama-3-8B-Instruct_prune_lora_awq"
model_id ="Meta-Llama-3-8B-Instruct_prune_lora_awq"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Meta-Llama-3-8B-Instruct_prune_lora_awq"

#### Llama-2-7b-chat-hf_prune_lora_awq

In [11]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Llama-2-7b-chat-hf_prune_lora_awq"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Llama-2-7b-chat-hf_prune_lora_awq"
model_id ="Llama-2-7b-chat-hf_prune_lora_awq"

## 8. GPTQ Models

#### Llama-3-8B-Instruct-prune-lora-gptq 4 Bit

In [21]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_lora_gptq/Llama_3_8B_Instruct-prune-lora-gptq_4bit"
model_id = "Llama-3-8B-Instruct-prune-lora-gptq_4bit"
tokenizer = "/home/thsch026/masterarbeit/models/generated/prune_lora_gptq/Llama_3_8B_Instruct-prune-lora-gptq_4bit"

#### Llama-3-8B-Instruct_gptq_4bit

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/gptq/Meta-Llama-3-8B-instruct_gptq_4bit"
model_id = "Llama-3-8B-Instruct_gptq_4bit"
tokenizer = "/home/thsch026/masterarbeit/models/generated/gptq/Meta-Llama-3-8B-instruct_gptq_4bit"

#### Llama-3-8B-Instruct_gptq_3bit

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/gptq/Meta-Llama-3-8B-instruct_gptq_3bit"
model_id = "Llama-3-8B-Instruct_gptq_3bit"
tokenizer = "/home/thsch026/masterarbeit/models/generated/gptq/Meta-Llama-3-8B-instruct_gptq_3bit"

#### Llama-3-8B-Instruct_gptq_2bit

In [12]:
model_loc = "/home/thsch026/masterarbeit/models/generated/gptq/Meta-Llama-3-8B-instruct_gptq_2bit"
model_id = "Llama-3-8B-Instruct_gptq_2bit"
tokenizer = "/home/thsch026/masterarbeit/models/generated/gptq/Meta-Llama-3-8B-instruct_gptq_2bit"

### Mistral 7B Instruct 4 Bit

In [10]:
model_loc = "/home/thsch026/masterarbeit/models/generated/gptq/Mistral-7B-Instruct-v0.2_gptq_4bit"
tokenizer = "/home/thsch026/masterarbeit/models/generated/gptq/Mistral-7B-Instruct-v0.2_gptq_4bit"
model_id = "Mistral 7B Instruct v0.2 gptq 4Bit"

### Mistral 7B Instruct 3 Bit

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/gptq/Mistral-7B-Instruct-v0.2_gptq_3bit"
tokenizer = "/home/thsch026/masterarbeit/models/generated/gptq/Mistral-7B-Instruct-v0.2_gptq_3bit"
model_id = "Mistral 7B Instruct v0.2 gptq 3-Bit"

### Mistral 7B Instruct 2 Bit

In [1]:
model_loc = "/home/thsch026/masterarbeit/models/generated/gptq/Mistral-7B-Instruct-v0.2_gptq_2bit"
tokenizer = "/home/thsch026/masterarbeit/models/generated/gptq/Mistral-7B-Instruct-v0.2_gptq_2bit"
model_id = "Mistral 7B Instruct v0.2 gptq 2Bit"

In [ ]:
TEST

In [9]:
model_loc = "/home/thsch026/masterarbeit/models/generated/lora/Llama-3-8B-Instruct_ms-marco/merged"
tokenizer = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "Llama 3 8B Instruct LoRA ms-marco merged"

In [8]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/Meta-Llama-3-8B-Instruct_qlora"
tokenizer = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "Meta Llama 3 8B Instruct qlora (17000 steps)"

## Inference Task

### Laden des Modells

In [6]:
print("\nLoading Model: ", model_id)
tokenizer = AutoTokenizer.from_pretrained(tokenizer)

quantization_config = BitsAndBytesConfig()
#quantization_config = BitsAndBytesConfig(bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_loc,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    #quantization_config=quantization_config,
    device_map='cuda'
)
print ("Loading COMPLETED")

#safetensors=True,
    #load_in_4bit=True,
    # Bei AWQ Modellen keine quantization config angeben


Loading Model:  llama-2-7b-chat_qlora


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading COMPLETED


### Inferenz mit Ergebnis Speicherung

In [ ]:
# Funktion für das Speichern der Werte

def append_data(result):
    
    with open(full_csv, mode='a', newline='') as file:
        full_writer = csv.writer(file)
        full_writer.writerow(result)
        
    with open(data_csv, mode='a', newline='') as file:
        data_writer = csv.writer(file)
        data_writer.writerow(result[0:4])

                
device = 'cuda'

question_list = ["Once upon a time", "Why should I eat healthy?", "Is there a second life after death?", "What is the most common injury?","Who is Konrad Zuse?"]

for question in question_list:
    prompt = f'''You are an helpful assistant. USER: {question} ASSISTANT:'''
    input_ids = tokenizer.encode(question, return_tensors='pt')
    # Änderung dr Zeile weil es Probleme mit der Ausführung gab (cuda/cpu anomalie)
    input_ids = tokenizer(question, return_tensors="pt").input_ids.to(device)
    input_ids.to('cuda')
    start_time = time.time()

    # Führe die Inferenz durch
    outputs = model.generate(input_ids, max_length=200)

    # Messe die Zeit nach der Inferenz
    end_time = time.time()

    # Berechne die Inferenz-Zeit
    inference_time = end_time - start_time

    # Dekodiere die Ausgabe
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Berchnung des Ergebnis
    #number_of_words = len(output_text.split())
    number_of_tokens = len(outputs[0])
    memory_usage = round((torch.cuda.memory_allocated() / (1024 **3)),2)
    token_per_second = number_of_tokens/inference_time
    result = [model_id,token_per_second, inference_time, memory_usage, question, output_text]
    print(result)
    append_data(result)
              
    
                                                                
    

    
    

## Inferenz Durchschnittswerte

In [7]:
# Funktion für das Speichern der Werte
from statistics import mean
def append_data(result):

    with open(full_csv, mode='a', newline='') as file:
        full_writer = csv.writer(file)
        full_writer.writerow(result)

def append_avg(result_avg):

    with open(data_csv, mode='a', newline='') as file:
        data_writer = csv.writer(file)
        data_writer.writerow(result_avg)

# Clean memory

device = 'cuda'
token_avg_list = []
memory_avg_list = []

# del(model)
# torch.cuda.empty_cache()

question_list = ["Once upon a time", "Why should I eat healthy?", "Is there a second life after death?", "What is the most common injury?","Who is Konrad Zuse?"]

for question in question_list:
    prompt = f'''You are an helpful assistant. USER: {question} ASSISTANT:'''
    input_ids = tokenizer.encode(question, return_tensors='pt')
    # Änderung dr Zeile weil es Probleme mit der Ausführung gab (cuda/cpu anomalie)
    input_ids = tokenizer(question, return_tensors="pt").input_ids.to(device)
    input_ids.to('cuda')
    start_time = time.time()

    # Führe die Inferenz durch
    outputs = model.generate(input_ids, max_length=200)

    # Messe die Zeit nach der Inferenz
    end_time = time.time()

    # Berechne die Inferenz-Zeit
    inference_time = end_time - start_time

    # Dekodiere die Ausgabe
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Berchnung des Ergebnis
    #number_of_words = len(output_text.split())
    number_of_tokens = len(outputs[0])
    memory_usage = round((torch.cuda.memory_allocated() / (1024 **3)),2)
    token_per_second = number_of_tokens/inference_time
    token_avg_list.append(token_per_second)
    memory_avg_list.append(memory_usage)
    result = [model_id,token_per_second, inference_time, memory_usage, question, output_text]
    print(result)
    append_data(result)

token_avg = mean(token_avg_list)
memory_avg = mean(memory_avg_list)
result_avg = [model_id, token_avg, memory_avg]
append_avg(result_avg)
print(result_avg)


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

### Beispiel von HF für AWQ Models

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = model_loc

# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=True, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

prompt = "Tell me about AI"
prompt_template=f'''{prompt}

'''

print("\n\n*** Generate:")

tokens = tokenizer(
    prompt_template,
    return_tensors='pt'
).input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=512
)

print("Output: ", tokenizer.decode(generation_output[0]))


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/em_german_mistral_v01-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Warum soll ich gesund essen?"
prompt_template=f'''Du bist ein hilfreicher Assistent. USER: {prompt} ASSISTANT:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))


In [7]:
del model # Specify variable
torch.cuda.empty_cache()